In [57]:
import random

# from rich import print

from ollama import Client
from ollama._types import ChatResponse
import ollama

def get_weather(city: str) -> str:
  """
  Get the current temperature for a city

  Args:
      city (str): The name of the city

  Returns:
      str: The current temperature
  """
  temperatures = list(range(-10, 35))

  temp = random.choice(temperatures)

  return f'The temperature in {city} is {temp}°C'


def get_weather_conditions(city: str) -> str:
  """
  Get the weather conditions for a city

  Args:
      city (str): The name of the city

  Returns:
      str: The current weather conditions
  """
  conditions = ['sunny', 'cloudy', 'rainy', 'snowy', 'foggy']
  return random.choice(conditions)


available_tools = {'get_weather': get_weather, 'get_weather_conditions': get_weather_conditions}

messages = [{'role': 'user', 'content': 'What is the weather like in London? What are the conditions in Toronto?'}]


# client = Client(
#   # Ollama Turbo
#   # host="https://ollama.com", headers={'Authorization': (os.getenv('OLLAMA_API_KEY'))}
# )
# model = 'gpt-oss:20b'
model = 'gemma3n-tools'
# gpt-oss can call tools while "thinking"
# a loop is needed to call the tools and get the results
while True:
  response: ChatResponse = ollama.chat(model=model, messages=messages, tools=[get_weather, get_weather_conditions])

  if response.message.content:
    print('Content: ')
    print(response.message.content + '\n')
  if response.message.thinking:
    print('Thinking: ')
    print(response.message.thinking + '\n')

  messages.append(response.message)

  if response.message.tool_calls:
    for tool_call in response.message.tool_calls:
      function_to_call = available_tools.get(tool_call.function.name)
      if function_to_call:
        result = function_to_call(**tool_call.function.arguments)
        print('Result from tool call name: ', tool_call.function.name, 'with arguments: ', tool_call.function.arguments, 'result: ', result + '\n')
        messages.append({'role': 'tool', 'content': result, 'tool_name': tool_call.function.name})
      else:
        print(f'Tool {tool_call.function.name} not found')
        messages.append({'role': 'tool', 'content': f'Tool {tool_call.function.name} not found', 'tool_name': tool_call.function.name})
  else:
    # no more tool calls, we can stop the loop
    break
    # Get user input and add to messages
    # user_input = input(">")
    # messages.append({'role': 'user', 'content': user_input})

Result from tool call name:  get_weather with arguments:  {'city': 'London'} result:  The temperature in London is 9°C

Result from tool call name:  get_weather with arguments:  {'city': 'Toronto'} result:  The temperature in Toronto is 23°C

Content: 
Okay, I have the weather in London and Toronto. 

London: 9°C
Toronto: 23°C

Is there anything else I can help you with?







In [56]:
tools=[get_weather, get_weather_conditions]
tools

[<function __main__.get_weather(city: str) -> str>,
 <function __main__.get_weather_conditions(city: str) -> str>]

In [47]:
import os
from litellm import completion
os.environ["OPENROUTER_API_KEY"] = "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"
os.environ["OPENROUTER_API_BASE"] = "" # [OPTIONAL] defaults to https://openrouter.ai/api/v1

messages=[{ "content": "Hello, how are you?","role": "user"}]
response = completion(
            model="openrouter/qwen/qwen3-14b:free",
            messages=messages,
        )

ModelResponse(id='gen-1757135038-tSAcu3UGbpEO5RkGUJC9', created=1757135038, model='qwen/qwen3-14b:free', object='chat.completion', system_fingerprint=None, choices=[Choices(finish_reason='stop', index=0, message=Message(content="Hello! I'm just a machine, so I don't have feelings, but I'm here and ready to help you with whatever you need! 😊 How are you doing today? Let me know if there's anything I can assist you with!", role='assistant', tool_calls=None, function_call=None, reasoning_content='\nOkay, the user greeted me with "Hello, how are you?" I should respond politely and warmly. Let me start with a friendly greeting. Since I\'m an AI, I don\'t have feelings, but I can express a positive attitude. I should mention that I\'m just a machine and don\'t experience emotions, but I\'m here to help. Then, I need to ask them how they\'re doing to keep the conversation going. Make sure the tone is cheerful and welcoming. Maybe add an emoji to make it more friendly. Check for any grammar is

In [41]:
# model="openrouter/qwen/qwen3-30b-a3b:free"
model="ollama/qwen3:8b"
import litellm
litellm.register_model(model_cost={
                "ollama_chat/qwen3:8b": { 
                  "supports_function_calling": True
                },
            })

{'ollama_chat/qwen3:8b': {'supports_function_calling': True}}

In [54]:
model="openrouter/qwen/qwen3-4b:free"

In [53]:
model

'openrouter/qwen/qwen3-30b-a3b:free'

In [55]:
import random
import json
from pprint import pprint
from litellm import completion
from tools import create_tools_from_functions
def get_weather(city: str) -> str:
  """
  Get the current temperature for a city

  Args:
      city (str): The name of the city

  Returns:
      str: The current temperature
  """
  temperatures = list(range(-10, 35))

  temp = random.choice(temperatures)

  return f'The temperature in {city} is {temp}°C'


def get_weather_conditions(city: str) -> str:
  """
  Get the weather conditions for a city

  Args:
      city (str): The name of the city

  Returns:
      str: The current weather conditions
  """
  conditions = ['sunny', 'cloudy', 'rainy', 'snowy', 'foggy']
  return random.choice(conditions)


available_tools = {'get_weather': get_weather, 'get_weather_conditions': get_weather_conditions}
tools = create_tools_from_functions(available_tools)
messages = [{'role': 'user', 'content': 'What is the weather like in London? What are the conditions in Toronto?'}]
print (tools)
# client = Client(
#   # Ollama Turbo
#   # host="https://ollama.com", headers={'Authorization': (os.getenv('OLLAMA_API_KEY'))}
# )
# model = 'gpt-oss:20b'
# model = 'gemma3n-tools'
# gpt-oss can call tools while "thinking"
# a loop is needed to call the tools and get the results
while True:
  response = completion(model=model, messages=messages, tools=tools)
  # pprint(response.choices[0].reasoning)
  pprint(response.choices[0])
  pprint(response.choices[0].message)
  
  # Access the message correctly
  message = response.choices[0].message
  
  if message.content:
    print('Content: ')
    print(message.content + '\n')
  if hasattr(message, 'thinking') and message.thinking:
    print('Thinking: ')
    print(message.thinking + '\n')

  messages.append(message)
  print(message)
  # Check for tool calls correctly
  if hasattr(message, 'tool_calls') and message.tool_calls:
    for tool_call in message.tool_calls:
      function_to_call = available_tools.get(tool_call.function.name)
      if function_to_call:
        try:
          # Parse the JSON arguments string
          arguments = json.loads(tool_call.function.arguments)
          result = function_to_call(**arguments)
          print('Result from tool call name: ', tool_call.function.name, 'with arguments: ', arguments, 'result: ', result + '\n')
          # Use tool role with tool_call_id as per OpenRouter documentation
          messages.append({
            'role': 'tool', 
            'content': result,
            'tool_call_id': tool_call.id
          })
        except (json.JSONDecodeError, TypeError) as e:
          print(f'Error parsing arguments for {tool_call.function.name}: {e}')
          messages.append({
            'role': 'tool', 
            'content': f"Error parsing arguments: {e}",
            'tool_call_id': tool_call.id
          })
        except Exception as e:
          print(f'Error calling {tool_call.function.name}: {e}')
          messages.append({
            'role': 'tool', 
            'content': f"Error calling function: {e}",
            'tool_call_id': tool_call.id
          })
      else:
        print(f'Tool {tool_call.function.name} not found')
        messages.append({
          'role': 'tool', 
          'content': f"Tool {tool_call.function.name} not found",
          'tool_call_id': tool_call.id
        })
  else:
    # no more tool calls, we can stop the loop
    break
    # Get user input and add to messages
    # user_input = input(">")
    # messages.append({'role': 'user', 'content': user_input})

[{'type': 'function', 'function': {'name': 'get_weather', 'description': 'Get the current temperature for a city', 'parameters': {'type': 'object', 'properties': {'city': {'type': 'string', 'description': 'Parameter: city'}}, 'required': ['city']}}}, {'type': 'function', 'function': {'name': 'get_weather_conditions', 'description': 'Get the weather conditions for a city', 'parameters': {'type': 'object', 'properties': {'city': {'type': 'string', 'description': 'Parameter: city'}}, 'required': ['city']}}}]
Choices(finish_reason='tool_calls', index=0, message=Message(content='', role='assistant', tool_calls=[ChatCompletionMessageToolCall(index=0, function=Function(arguments='{"city": "London"}', name='get_weather'), id='chatcmpl-tool-bda756d15cde425b826e32c5882cf41e', type='function'), ChatCompletionMessageToolCall(index=1, function=Function(arguments='{"city": "Toronto"}', name='get_weather_conditions'), id='chatcmpl-tool-0e6c9620cf614126ae7f212d3fc09c56', type='function')], function_ca

In [44]:
pprint(messages)

[{'content': 'What is the weather like in London? What are the conditions in '
             'Toronto?',
  'role': 'user'},
 Message(content=None, role='assistant', tool_calls=[{'function': {'arguments': '{"city": "London"}', 'name': 'get_weather'}, 'id': 'call_68b2efc1-b876-4d00-b51c-c4f45f3650f0', 'type': 'function'}], function_call=None, provider_specific_fields=None),
 {'content': 'The temperature in London is 16°C',
  'role': 'tool',
  'tool_call_id': 'call_68b2efc1-b876-4d00-b51c-c4f45f3650f0'},
 Message(content=None, role='assistant', tool_calls=[{'function': {'arguments': '{"city": "London"}', 'name': 'get_weather'}, 'id': 'call_03478066-ef7e-4062-bfd1-02ad07494b74', 'type': 'function'}], function_call=None, provider_specific_fields=None),
 {'content': 'The temperature in London is 16°C',
  'role': 'tool',
  'tool_call_id': 'call_03478066-ef7e-4062-bfd1-02ad07494b74'},
 Message(content=None, role='assistant', tool_calls=[{'function': {'arguments': '{"city": "London"}', 'name': '

In [16]:
import litellm
print (litellm.supports_function_calling(model="openrouter/qwen/qwen3-14b:free"))

False


In [27]:
pprint  (messages[-1].content)

("Here's the weather information:\n"
 '\n'
 '**London**  \n'
 '- **Temperature:** 32°C  \n'
 '- **Conditions:** Cloudy  \n'
 '\n'
 '**Toronto**  \n'
 '- **Conditions:** Cloudy  \n'
 '\n'
 "Note: Toronto's temperature was not provided in the tool response. Let me "
 "know if you'd like additional details!")


In [59]:
from ollama import Client
client = Client(
  # Ollama Turbo
  host="https://ollama-host.loca.lt/", 
  # headers={'Authorization': (os.getenv('OLLAMA_API_KEY'))}
)

client.chat??

Signature:
client.chat(
    model: str = '',
    messages: Optional[Sequence[Union[Mapping[str, Any], ollama._types.Message]]] = None,
    *,
    tools: Optional[Sequence[Union[Mapping[str, Any], ollama._types.Tool, Callable]]] = None,
    stream: bool = False,
    think: Union[bool, Literal['low', 'medium', 'high'], NoneType] = None,
    format: Union[Literal['', 'json'], dict[str, Any], NoneType] = None,
    options: Union[Mapping[str, Any], ollama._types.Options, NoneType] = None,
    keep_alive: Union[float, str, NoneType] = None,
) -> Union[ollama._types.ChatResponse, collections.abc.Iterator[ollama._types.ChatResponse]]
Source:   
  def chat(
    self,
    model: str = '',
    messages: Optional[Sequence[Union[Mapping[str, Any], Message]]] = None,
    *,
    tools: Optional[Sequence[Union[Mapping[str, Any], Tool, Callable]]] = None,
    stream: bool = False,
    think: Optional[Union[bool, Literal['low', 'medium', 'high']]] = None,
    format: Optional[Union[Literal['', 'json'], 